## Bibliotecas

In [18]:
# !pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org streamlit_folium

In [1]:
# 1.Bibliotecas
import pandas as pd
import inflection
import sweetviz as sv
import numpy as np
import folium
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
from collections import namedtuple
from folium.plugins import MarkerCluster


c:\Users\mluciano\anaconda3\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
df[20]

KeyError: 20

## Importação da base de dados

In [4]:
# 2. Importando a base de dados
df = pd.read_csv('dataset/zomato.csv')
df1 = pd.read_csv('dataset/zomato.csv')

## Limpezas e Funções

#### Funções

In [5]:
df.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes'],
      dtype='object')

In [6]:
# 3. Funções e dicionários

# 3.1 Criação do nome das Cores
COLORS = {
"3F7E00": "darkgreen",
"5BA829": "green",
"9ACD32": "lightgreen",
"CDD614": "orange",
"FFBA00": "red",
"CBCBC8": "darkred",
"FF7800": "darkred",
}

def color_name(color_code):
    return COLORS[color_code]

# 3.2 Preenchimento do nome dos países
COUNTRIES = {
1: "India",
14: "Australia",
30: "Brazil",
37: "Canada",
94: "Indonesia",
148: "New Zeland",
162: "Philippines",
166: "Qatar",
184: "Singapure",
189: "South Africa",
191: "Sri Lanka",
208: "Turkey",
214: "United Arab Emirates",
215: "England",
216: "United States of America",
}

def country_name(country_id):
    return COUNTRIES[country_id]

# 3.3 Criação do Tipo de Categoria de Comida
def create_price_tye(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"

# 3.4 Renomear as colunas do DataFrame    
def rename_columns(dataframe):
    df = dataframe.copy()
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(" ", "")
    cols_old = list(df.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new

    return df

# 3.5 Total de tipos de culinárias disponíveis
def fnc_tot_tp_culinaria( df ):
    vListaCuisines = ( set( df['cuisines'] ) )
    vListaTotalCuisine = []
    
    for vCuisine in vListaCuisines:
        vTeste = vCuisine.split(',')
        # print(vTeste)
        for vPalavra in vTeste:
            vPalavra = vPalavra.strip()
            # print(vPalavra)
            if vPalavra in vListaTotalCuisine:
                None
            
            else:
                vListaTotalCuisine.append( vPalavra )

    df_aux = pd.DataFrame(vListaTotalCuisine)
    
    return df_aux.count()

# 3.6 Calculos por país
def fnc_calc_per_country( flag, tp_calc, df ):
    v01 = 0
    v02 = 0
    vList01 = []
    vList02 = []

    # Quantidade de cidades registradas por pais
    if flag == 1:
        for i in df['country_code'].unique():
            df_aux01 = df.loc[ ( df['country_code'] == i ), [ 'city' ] ]
            v01 = len( df_aux01[ 'city' ].unique() )

            vList01.append( country_name( i ) )
            vList02.append( v01 )
            data = { 'countries': vList01,
                     'num_cities' : vList02 }

            df_return = pd.DataFrame( data )

            if v01 > v02:
                v02 = v01
                vMaiorPais = i

        if tp_calc == 'grafico':
            return df_return
        
        elif tp_calc == 'maior':
            return country_name( vMaiorPais )

    # Quantidade de restaurantes registrados por pais    
    elif flag == 2:
        for i in df['country_name'].unique():
            df_aux01 = df.loc[ ( df['country_name'] == i ), [ 'restaurant_id' ] ]
            v01 = len( df_aux01[ 'restaurant_id' ].unique() )

            vList01.append( i )
            vList02.append( v01 )

            data = { 'countries': vList01,
                     'num_restaurants' : vList02 }

            data = pd.DataFrame( data )

            if v01 > v02:
                v02 = v01
                vMaiorPais = i

        if tp_calc == 'grafico':

            data.columns = [ 'Países', 'Quantidade de Restaurantes' ]
            fig = go.Figure()
            fig = ( px.bar(data,
                         text='Quantidade de Restaurantes',
                        x='Países', y='Quantidade de Restaurantes').update_traces( marker_color='#66b3ff' )
                                                                    .update_layout( plot_bgcolor='white' )
                                                                    .update_xaxes( mirror=True,
                                                                                    ticks='outside',
                                                                                    showline=True,
                                                                                    linecolor='black',
                                                                                    gridcolor='lightgrey' )
                                                                    .update_yaxes( mirror=True,
                                                                                    ticks='outside',
                                                                                    showline=True,
                                                                                    linecolor='black',
                                                                                    gridcolor='lightgrey')
                                                                    )
            return fig
        
        elif tp_calc == 'maior':
            return vMaiorPais

    # Quantidade de restaurantes com nivel de preço igual a 4 (gourmet)    
    elif flag == 3:
        for i in df['country_code'].unique():
            df_aux01 = df.loc[ ( ( df['country_code'] == i) & ( df[ 'price_range' ] == 4 ) ), [ 'restaurant_id' ] ]
            v01 = len( df_aux01[ 'restaurant_id' ].unique() )

            vList01.append( country_name( i ) )
            vList02.append( v01 )

            data = { 'countries': vList01,
                     'num_restaurants' : vList02 }

            df_return = pd.DataFrame( data )

            if v01 > v02:
                v02 = v01
                vMaiorPais = i

        if tp_calc == 'grafico':
            return df_return
        
        elif tp_calc == 'maior':
            return country_name( vMaiorPais )
    
    # Quantidade de culinarias unicas
    if flag == 4:
        for i in df['country_code'].unique():
            df_aux01 = df.loc[ ( df['country_code'] == i ), [ 'cuisines' ] ]
            v01 = len( df_aux01[ 'cuisines' ].unique() )

            vList01.append( country_name( i ) )
            vList02.append( v01 )
            data = { 'countries': vList01,
                     'num_cuisines' : vList02 }

            df_return = pd.DataFrame( data )

            if v01 > v02:
                v02 = v01
                vMaiorPais = i

        if tp_calc == 'grafico':
            return df_return
        
        elif tp_calc == 'maior':
            return country_name( vMaiorPais )
        
    # Soma de avaliações por país
    if flag == 5:
        for i in df['country_code'].unique():
            df_aux01 = df.loc[ ( df['country_code'] == i ), [ 'votes' ] ]
            v01 = df_aux01[ 'votes' ].sum()

            vList01.append( country_name( i ) )
            vList02.append( v01 )
            data = { 'countries': vList01,
                     'qt_votes' : vList02 }

            df_return = pd.DataFrame( data )

            if v01 > v02:
                v02 = v01
                vMaiorPais = i

        if tp_calc == 'grafico':
            return df_return
        
        elif tp_calc == 'maior':
            return country_name( vMaiorPais )
        
    # Quantidade de restaurantes que fazem entrega
    if flag == 6:
        for i in df['country_code'].unique():
            df_aux01 = df.loc[ ( ( df['country_code'] == i) & ( df[ 'is_delivering_now' ] == 1 ) ), : ]
            v01 = len( df_aux01 )

            vList01.append( country_name( i ) )
            vList02.append( v01 )
            data = { 'countries': vList01,
                     'qt_delivery' : vList02 }

            df_return = pd.DataFrame( data )

            if v01 > v02:
                v02 = v01
                vMaiorPais = i

        if tp_calc == 'grafico':
            return df_return
        
        elif tp_calc == 'maior':
            return country_name( vMaiorPais )
        
    # Quantidade de restaurantes que aceitam reservas
    if flag == 7:
        for i in df['country_code'].unique():
            df_aux01 = df.loc[ ( ( df['country_code'] == i) & ( df[ 'has_table_booking' ] == 1 ) ), : ]
            v01 = len( df_aux01 )

            vList01.append( country_name( i ) )
            vList02.append( v01 )
            data = { 'countries': vList01,
                     'qt_booking' : vList02 }

            df_return = pd.DataFrame( data )

            if v01 > v02:
                v02 = v01
                vMaiorPais = i

        if tp_calc == 'grafico':
            return df_return
        
        elif tp_calc == 'maior':
            return country_name( vMaiorPais )

### Limpezas

In [7]:
# 1. Renomeando as colunas
df = rename_columns( df )

# 2. Removenco as linhas em branco
# cuisines
df.dropna( subset=[ 'cuisines' ], inplace=True )

# 3. Criando a coluna cuisine_category

# 4. Criando a coluna country_name
df[ 'country_name' ] = df[ 'country_code' ].apply( lambda x: country_name( x ) )

# 5. Categorizando os restaurantes para somente um tipo de culinária(cuisines)
df['cuisines'] = df['cuisines'].astype(str)
df['cuisines'] = df.loc[ :, 'cuisines' ].apply( lambda x: x.split( ',' )[0] )

# 6. Removendo Cuisines do tipo Mineira e Drinks Only
df = df.loc[( df['cuisines'] != 'Drinks Only') & ( df['cuisines'] != 'Mineira' ) & ( df['cuisines'] != 'NaN' ), : ]

# 7. Removendo linhas duplicadas
df = df.drop_duplicates()

In [15]:
vRestUnique = df[ 'country_name' ].unique()
vRestUnique

array(['Philippines', 'Brazil', 'Australia', 'United States of America',
       'Canada', 'Singapure', 'United Arab Emirates', 'India',
       'Indonesia', 'New Zeland', 'England', 'Qatar', 'South Africa',
       'Sri Lanka', 'Turkey'], dtype=object)

## Respostas das perguntas de negócio

### Geral

##### 1. Quantos restaurantes únicos estão registrados?

In [239]:
vRestUnique = df[ 'restaurant_id' ].nunique()
vRestUnique

6927

##### 2. Quantos países únicos estão registrados?

In [240]:
vCountryUnique = df[ 'country_code' ].nunique()
vCountryUnique

15

#### 3. Quantas cidades únicas estão registradas?

In [241]:
vCityUnique = df[ 'city' ].nunique()
vCityUnique

125

#### 4. Qual o total de avaliações feitas?

In [242]:
vCountVote = df[ 'votes' ].sum()
vCountVote = '{:_}'.format( df[ 'votes' ].sum() ).replace( '.', '.' ).replace( '_', '.' )
vCountVote 

'4.194.530'

#### 5. Qual o total de tipos de culinária registrados?

In [243]:
vTotalCuisines = fnc_tot_tp_culinaria( df )
vTotalCuisines

0    163
dtype: int64

### País

##### 1. Qual o nome do país que possui mais cidades registradas?

In [7]:
print( fnc_calc_per_country( 1, 'maior', df ) )
print( fnc_calc_per_country( 1, 'grafico', df ) )

India
                   countries  num_cities
0                Philippines          12
1                     Brazil           3
2                  Australia           3
3   United States of America          22
4                     Canada           3
5                  Singapure           1
6       United Arab Emirates           4
7                      India          49
8                  Indonesia           3
9                 New Zeland           4
10                   England           5
11                     Qatar           1
12              South Africa          12
13                 Sri Lanka           1
14                    Turkey           2


##### 2. Qual o nome do país que possui mais restaurantes registrados?

In [ ]:
print( fnc_calc_per_country( 2, 'maior', df ) )
print( fnc_calc_per_country( 2, 'grafico', df ) )

##### 3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4 registrados?

In [ ]:
print( fnc_calc_per_country( 3, 'maior', df ) )
print( fnc_calc_per_country( 3, 'grafico', df ) )

##### 4. Qual o nome do país que possui a maior quantidade de tipos de culinária distintos?

In [ ]:
print( fnc_calc_per_country( 4, 'maior', df ) )
print( fnc_calc_per_country( 4, 'grafico', df ) )

##### 5. Qual o nome do país que possui a maior quantidade de avaliações feitas?

In [ ]:
print( fnc_calc_per_country( 5, 'maior', df ) )
print( fnc_calc_per_country( 5, 'grafico', df ) )

##### 6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?

In [ ]:
print( fnc_calc_per_country( 6, 'maior', df ) )
print( fnc_calc_per_country( 6, 'grafico', df ) )

##### 7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?

In [ ]:
print( fnc_calc_per_country( 7, 'maior', df ) )
print( fnc_calc_per_country( 7, 'grafico', df ) )

##### 8. Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?

In [196]:
df_aux = df.loc[ :, [ 'votes', 'country_name' ] ].groupby( [ 'country_name' ] ).mean().sort_values( [ 'votes' ], ascending=False ).reset_index()

df_aux['country_name'][0]

'Indonesia'

##### 9. Qual o nome do país que possui, na média, a maior nota média registrada?

In [195]:
df_aux = df.loc[ :, [ 'aggregate_rating', 'country_name' ] ].groupby( [ 'country_name' ] ).mean().sort_values( [ 'aggregate_rating' ], ascending=False ).reset_index()

df_aux['country_name'][0]

'Indonesia'

##### 10. Qual o nome do país que possui, na média, a menor nota média registrada?

In [193]:
df_aux = df.loc[ :, [ 'aggregate_rating', 'country_name' ] ].groupby( [ 'country_name' ] ).mean().sort_values( [ 'aggregate_rating' ], ascending=True ).reset_index()

# country_name( df_aux['country_code'][0] )
df_aux['country_name'][0]

'Brazil'

##### 11. Qual a média de preço de um prato para dois por país?

In [ ]:
df.loc[ :, [ 'average_cost_for_two', 'country_name' ] ].groupby( [ 'country_name' ] ).mean().sort_values( 'average_cost_for_two', ascending=False ).reset_index()

### Cidade

##### 1. Qual o nome da cidade que possui mais restaurantes registrados?

In [18]:
df_aux = df.loc[ :, [ 'restaurant_id', 'city', 'country_name' ] ].groupby( [ 'city', 'country_name' ] ).count().sort_values( ['restaurant_id'], ascending = False ).reset_index().head(10)
df_aux.columns = [ 'Cidade', 'País', 'Quantidade de Restaurantes' ]
df_aux

,Cidade,País,Quantidade de Restaurantes
0,Nagpur,India,99
1,Surat,India,95
2,Birmingham,England,95
3,Abu Dhabi,United Arab Emirates,95
4,Doha,Qatar,94
5,Goa,India,94
6,Cape Town,South Africa,94
7,Patna,India,93
8,Rio de Janeiro,Brazil,92
9,Aurangabad,India,92


In [11]:
df_aux = df.loc[ :, [ 'restaurant_id', 'city' ] ].groupby( [ 'city' ] ).count().sort_values( ['restaurant_id'], ascending = False ).reset_index().head(10)
df_aux['city'][0]

df_aux

,city,restaurant_id
0,Nagpur,99
1,Surat,95
2,Birmingham,95
3,Abu Dhabi,95
4,Doha,94
5,Goa,94
6,Cape Town,94
7,Patna,93
8,Rio de Janeiro,92
9,Aurangabad,92


##### 2. Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?

In [210]:
df.loc[ ( df[ 'aggregate_rating' ] > 4 ), [ 'restaurant', 'city' ] ].groupby( [ 'city' ] ).count().sort_values( [ 'restaurant' ], ascending=False ).reset_index().head(1)
df_aux['city'][0]

'Nagpur'

##### 3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?

In [213]:
df_aux = df.loc[ ( df[ 'aggregate_rating' ] < 2.5 ), [ 'restaurant', 'city' ] ].groupby( [ 'city' ] ).count().sort_values( [ 'restaurant' ], ascending=False ).reset_index().head(1)
df_aux['city'][0]

'Gangtok'

##### 4. Qual o nome da cidade que possui o maior valor médio de um prato para dois?

In [214]:
df_aux = df.loc[ :, [ 'average_cost_for_two', 'city' ] ].groupby( [ 'city' ] ).mean().sort_values( [ 'average_cost_for_two' ], ascending=False ).reset_index().head(1)
df_aux['city'][0]

'Adelaide'

##### 5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?

In [221]:
df_aux01 = df.loc[ :, ['country_name', 'city', 'cuisines']]
df_aux01 = df_aux01.groupby( ['country_name', 'city'] ).nunique().sort_values( ['country_name', 'cuisines'], ascending=False ).reset_index()
df_aux01

,country_name,city,cuisines
0,United States of America,Houston,30
1,United States of America,Philadelphia,29
2,United States of America,Portland,28
3,United States of America,Seattle,27
4,United States of America,San Francisco,26
...,...,...,...
120,Brazil,Brasília,26
121,Brazil,Rio de Janeiro,24
122,Australia,Perth,29
123,Australia,Adelaide,25


In [ ]:
df_aux = []
lst_tst = []
lst_tst = []
vTotalCuisines = 0
lst_cuisines = df[ 'cuisines' ].unique()
lst_city = df[ 'city' ].unique()
countries = []
cities = []
cuisines = []
vMaiorCidade = None

for row in lst_city:
    df_aux = df.loc[ ( df['city'] == row ), : ]
    for row2 in df_aux['cuisines']:
        # print(row2)
        if row2 not in lst_tst:
            lst_tst.append(row2)

    if len( lst_tst ) > vTotalCuisines:    
        vTotalCuisines = len( lst_tst )
        vMaiorCidade = row
    print(row)
    print(vTotalCuisines)
    
    countries.append( df_aux['country_name'] )
    cities.append(row)
    cuisines.append(vTotalCuisines)
    df01 = pd.DataFrame({'country_name' : countries,
                         'city' : cities,
                         'cuisines' : cuisines})

df01

In [ ]:
df_aux = []
lst_tst = []
vMaiorCidade = None
vTotalCuisines = 0
lst_cuisines = df[ 'cuisines' ].unique()
lst_city = df[ 'city' ].unique()

for row in lst_city:
    df_aux = df.loc[ ( df['city'] == row ), : ]
    # print(row)
    for row2 in df_aux['cuisines']:
        # print(row2)
        if row2 not in lst_tst:
            lst_tst.append(row2)

    if len( lst_tst ) > vTotalCuisines:    
        vTotalCuisines = len( lst_tst )
        vMaiorCidade = row
        # print(vTotalCuisines)

    # print(vTotalCuisines)

print( vMaiorCidade )

##### 6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?

In [27]:
df_aux = df.loc[ ( df[ 'has_table_booking' ] == 1 ), [ 'restaurant_id', 'city' ] ].groupby( [ 'city' ] ).count().sort_values( [ 'restaurant_id' ], ascending=False ).reset_index().head(1)
df_aux[ 'city' ][0]

'Bangalore'

##### 7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?

In [100]:
df_aux = df.loc[ ( df[ 'is_delivering_now' ] == 1 ), [ 'restaurant_id', 'city' ] ].groupby( [ 'city' ] ).count().sort_values( [ 'restaurant_id' ], ascending=False ).reset_index().head(1)
df_aux[ 'city' ][0]

'Aurangabad'

##### 8. Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?

In [101]:
df_aux = df.loc[ ( df[ 'has_online_delivery' ] == 1 ), [ 'restaurant_id', 'city' ] ].groupby( [ 'city' ] ).count().sort_values( [ 'restaurant_id' ], ascending=False ).reset_index().head(1)
df_aux[ 'city' ][0]

'Abu Dhabi'

### Restaurantes

##### 1. Qual o nome do restaurante que possui a maior quantidade de avaliações?

In [103]:
df_aux = df.loc[ :, [ 'votes', 'restaurant_name' ] ].groupby( [ 'restaurant_name' ] ).sum().sort_values( [ 'votes' ], ascending=False ).reset_index().head(1)
df_aux[ 'restaurant_name' ][0]

"Domino's Pizza"

##### 2. Qual o nome do restaurante com a maior nota média?

In [104]:
df_aux = df.loc[ :, [ 'aggregate_rating', 'restaurant_name' ] ].groupby( [ 'restaurant_name' ] ).mean().sort_values( [ 'aggregate_rating' ], ascending=False ).reset_index().head(1)
df_aux[ 'restaurant_name' ][0]

'Andina Restaurant'

##### 3. Qual o nome do restaurante que possui o maior valor de uma prato para duas pessoas?

In [107]:
df_aux = df.loc[ :, [ 'average_cost_for_two', 'restaurant_name' ] ].sort_values( [ 'average_cost_for_two' ], ascending=False ).reset_index().head(1)
df_aux['restaurant_name'][0]

"d'Arry's Verandah Restaurant"

##### 4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?


In [120]:
df_aux = (df.loc[ ( df[ 'cuisines' ] == 'Brazilian' ), [ 'aggregate_rating', 'restaurant_name' ] ].groupby( [ 'restaurant_name' ] )
                                                                                                  .mean()
                                                                                                  .sort_values( [ 'aggregate_rating' ] )
                                                                                                  .reset_index()
                                                                                                  .head(1) )
df_aux[ 'restaurant_name' ][0]

'Restaurante dos Amigos'

##### 5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?


In [121]:
df_aux = ( df.loc[ ( df[ 'cuisines' ] == 'Brazilian' ) & (( df[ 'country_name' ] == 'Brazil' )), [ 'aggregate_rating', 'restaurant_name' ] ].groupby( [ 'restaurant_name' ] )
                                                                                                                                            .mean()
                                                                                                                                            .sort_values( [ 'aggregate_rating' ], ascending=False )
                                                                                                                                            .reset_index()
                                                                                                                                            .head(1) )
df_aux[ 'restaurant_name' ][0]

'Braseiro da Gávea'

##### 6. Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?


In [135]:
df_pedido_online = df.loc[ ( df[ 'has_online_delivery' ] == 1 ), [ 'restaurant_id', 'restaurant_name' ] ].sort_values( [ 'restaurant_id' ] ).reset_index()
print( df_pedido_online )

df_maior_media = ( df.loc[ :, [ 'votes', 'restaurant_name' ] ].groupby( [ 'restaurant_name' ] )
                                                              .mean()
                                                              .sort_values( [ 'votes' ], ascending=False )
                                                              .reset_index() )
print( df_maior_media )

      index  restaurant_id      restaurant_name
0      3760            549           Dana Choga
1      3660           1345    The Yellow Chilli
2      5085           2008          Hira Sweets
3      5037           4751  Chili's Grill & Bar
4      5038           4751  Chili's Grill & Bar
...     ...            ...                  ...
2648   5174       18994548   V. Krishnan Bakery
2649   5119       18997447   Adarsh Veg Kitchen
2650   5115       18997511      Jain Restaurant
2651   5748       19003551             Icekraft
2652   4733       19008239     HAS South Bombay

[2653 rows x 3 columns]
                     restaurant_name         votes
0                               Toit  15270.000000
1                           Truffles  14984.000000
2                           Bawarchi  13903.666667
3                   Hauz Khas Social  13627.000000
4     Shah Ghouse Hotel & Restaurant  11836.000000
...                              ...           ...
5894              Coco Bambú Anhembi      

##### 7. Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?


In [137]:
df_reserva = df.loc[ ( df[ 'has_table_booking' ] == 1 ), [ 'restaurant_id', 'restaurant_name' ] ].sort_values( [ 'restaurant_id' ] ).reset_index()
print( df_reserva )

df_maior_media = ( df.loc[ :, [ 'average_cost_for_two', 'restaurant_name' ] ].groupby( [ 'restaurant_name' ] )
                                                                             .mean()
                                                                             .sort_values( [ 'average_cost_for_two' ], ascending=False )
                                                                             .reset_index() )
print( df_maior_media )

     index  restaurant_id      restaurant_name
0     5075           8369          Garage Inc.
1     3758           8913     Pirates of Grill
2     3759           8913     Pirates of Grill
3     5435          10987            Cafe 1730
4     4362          20842      Barbeque Nation
..     ...            ...                  ...
463   3337       18938720       Coal Barbecues
464   3035       18945224  Secret Of Bangalore
465   4446       18951931        The Centurion
466   3245       18972215              Baithak
467   4447       19000106  The Drowning Street

[468 rows x 3 columns]
                              restaurant_name  average_cost_for_two
0                d'Arry's Verandah Restaurant            25000017.0
1                      The Café - Hotel Mulia             1200000.0
2                        GIA Restaurant & Bar              700000.0
3                                        SKYE              600000.0
4                         Shabu-Shabu Shaburi              600000.0
... 

##### 8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americana (BBQ)?

In [142]:
df_japa_eua = ( df.loc[ ( df[ 'country_name' ] == 'United States of America' ) & ( df[ 'cuisines' ] == 'Japanese' ), [ 'average_cost_for_two', 'restaurant_name' ] ].groupby( [ 'restaurant_name' ] )
                                                                                                                                                                    .mean()
                                                                                                                                                                    .sort_values( [ 'average_cost_for_two' ], ascending=False )
                                                                                                                                                                    .reset_index()
                                                                                                                                                                    .head(10) )
print( df_japa_eua )

df_cost_bbq = ( df.loc[ ( df[ 'cuisines' ] == 'BBQ' ), [ 'average_cost_for_two', 'restaurant_name' ] ].groupby( [ 'restaurant_name' ] )
                                                                                                      .mean()
                                                                                                      .sort_values( [ 'average_cost_for_two' ], ascending=False )
                                                                                                      .reset_index()
                                                                                                      .head(10) )
print( df_cost_bbq )

              restaurant_name  average_cost_for_two
0                        Nobu                 250.0
1                    Morimoto                 120.0
2         Roka Akor - Chicago                 100.0
3                        Uchi                  80.0
4                Bamboo Sushi                  75.0
5  Shibui Japanese Restaurant                  75.0
6                   Sushi Den                  70.0
7                        Raku                  70.0
8                         Pod                  70.0
9      SUSHISAMBA - Las Vegas                  70.0
             restaurant_name  average_cost_for_two
0      Magal Mapogalmegi BBQ              300000.0
1             The Indi Grill                1500.0
2           The Big Barbeque                1500.0
3  AB's - Absolute Barbecues                1400.0
4         Universal Barbeque                1200.0
5         Fresh Off The Boat                 900.0
6     Dindigul Thalappakatti                 800.0
7                  B

### Tipos de Culinária

##### 1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a maior média de avaliação?


In [255]:
df_aux = ( df.loc[ ( df[ 'cuisines' ] == 'Italian' ), [ 'aggregate_rating', 'restaurant_name' ] ].groupby( [ 'restaurant_name' ] )
                                                                                        .mean()
                                                                                        .sort_values( [ 'aggregate_rating' ], ascending=False )
                                                                                        .reset_index()
                                                                                        .head(1) )
df_aux[ 'restaurant_name' ][0]

'Central Grocery'

In [275]:
df.loc[ ( df['restaurant_name'] == 'Darshan' ), : ]


,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,has_table_booking,has_online_delivery,is_delivering_now,switch_to_order_menu,price_range,aggregate_rating,rating_color,rating_text,votes,country_name
5398,18732937,Darshan,1,Pune,"Amar Business Park, Baner, Pune",Baner,"Baner, Pune",73.767019,18.568732,Italian,...,0,1,1,0,2,4.6,3F7E00,Excellent,1657,India
5408,10021,Darshan,1,Pune,"759/60, Prabhat Road, Opposite Indian Bank, De...",Deccan Gymkhana,"Deccan Gymkhana, Pune",73.840957,18.514017,Italian,...,0,1,1,0,2,4.9,3F7E00,Excellent,3106,India


In [269]:
df_aux = ( df.loc[ ( df[ 'cuisines' ] == 'Italian' ), [ 'aggregate_rating', 'restaurant_name', 'restaurant_id' ] ].groupby( [ 'restaurant_name', 'restaurant_id' ] )
                                                                                                                    .mean()
                                                                                                                    .sort_values( [ 'aggregate_rating', 'restaurant_id' ], ascending=[ False, True ] )
                                                                                                                    .reset_index()
                                                                                                                    .head(15) )
df_aux

,restaurant_name,restaurant_id,aggregate_rating
0,Darshan,10021,4.9
1,Cafe Del Sol Classico,6501298,4.9
2,Ombra,7100171,4.9
3,Celino's,7700796,4.9
4,Andre's Cucina & Polenta Bar,16587684,4.9
5,Di Rienzo Grocery & Deli,16663419,4.9
6,Chicago Pizza & Oven Grinder Company,16733182,4.9
7,Regina Pizzeria,16797555,4.9
8,Bottega Louie,16831684,4.9
9,Perricone's Marketplace & Café,16923457,4.9


##### 2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?


In [147]:
df_aux = ( df.loc[ ( df[ 'cuisines' ] == 'Italian' ), [ 'aggregate_rating', 'restaurant_name' ] ].groupby( [ 'restaurant_name' ] )
                                                                                        .mean()
                                                                                        .sort_values( [ 'aggregate_rating' ] )
                                                                                        .reset_index()
                                                                                        .head(1) )
df_aux[ 'restaurant_name' ][0]

'La Bocca Bar e Trattoria'

##### 3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?


In [270]:
df_aux = ( df.loc[ ( df[ 'cuisines' ] == 'American' ), [ 'aggregate_rating', 'restaurant_name', 'restaurant_id' ] ].groupby( [ 'restaurant_id', 'restaurant_name' ] )
                                                                                                                   .mean()
                                                                                                                   .sort_values( [ 'aggregate_rating', 'restaurant_id' ], ascending = [ False, True ] )
                                                                                                                   .reset_index()
                                                                                                                   .head(50) )
df_aux[ 'restaurant_name' ][0]

'Burger & Lobster'

##### 4. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?


In [174]:
df_aux = ( df.loc[ ( df[ 'cuisines' ] == 'American' ), [ 'aggregate_rating', 'restaurant_name', 'restaurant_id' ] ].groupby( [ 'restaurant_name', 'restaurant_id' ] )
                                                                                                                   .mean()
                                                                                                                   .sort_values( [ 'aggregate_rating', 'restaurant_id' ], ascending = True )
                                                                                                                   .reset_index()
                                                                                                                   .head(50) )
df_aux[ 'restaurant_name' ][0]

'Alston Bar & Beef'

##### 5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a maior média de avaliação?


In [271]:
df_aux = ( df.loc[ ( df[ 'cuisines' ] == 'Arabian' ), [ 'aggregate_rating', 'restaurant_name', 'restaurant_id' ] ].groupby( [ 'restaurant_name', 'restaurant_id' ] )
                                                                                                                  .mean()
                                                                                                                  .sort_values( [ 'aggregate_rating', 'restaurant_id' ], ascending = [ False, True ] )
                                                                                                                  .reset_index()
                                                                                                                  .head(50) )
df_aux[ 'restaurant_name' ][0]

'Mandi@36'

##### 6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a menor média de avaliação?


In [180]:
df_aux = ( df.loc[ ( df[ 'cuisines' ] == 'Arabian' ), [ 'aggregate_rating', 'restaurant_name', 'restaurant_id' ] ].groupby( [ 'restaurant_name', 'restaurant_id' ] )
                                                                                                                  .mean()
                                                                                                                  .sort_values( [ 'aggregate_rating', 'restaurant_id' ], ascending = True )
                                                                                                                  .reset_index()
                                                                                                                  .head(50) )
df_aux[ 'restaurant_name' ][0]

'Raful'

##### 7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a maior média de avaliação?


In [181]:
df_aux = ( df.loc[ ( df[ 'cuisines' ] == 'Japanese' ), [ 'aggregate_rating', 'restaurant_name', 'restaurant_id' ] ].groupby( [ 'restaurant_name', 'restaurant_id' ] )
                                                                                                                   .mean()
                                                                                                                   .sort_values( [ 'aggregate_rating', 'restaurant_id' ], ascending = [ False, True ] )
                                                                                                                   .reset_index()
                                                                                                                   .head(50) )
df_aux[ 'restaurant_name' ][0]

'Sushi Samba'

##### 8. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a menor média de avaliação?


In [182]:
df_aux = ( df.loc[ ( df[ 'cuisines' ] == 'Japanese' ), [ 'aggregate_rating', 'restaurant_name', 'restaurant_id' ] ].groupby( [ 'restaurant_name', 'restaurant_id' ] )
                                                                                                                   .mean()
                                                                                                                   .sort_values( [ 'aggregate_rating', 'restaurant_id' ], ascending = True )
                                                                                                                   .reset_index()
                                                                                                                   .head(50) )
df_aux[ 'restaurant_name' ][0]

'Banzai Sushi'

##### 9. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a maior média de avaliação?


In [186]:
df_aux = ( df.loc[ ( df[ 'cuisines' ] == 'Home-made' ), [ 'aggregate_rating', 'restaurant_name', 'restaurant_id' ] ].groupby( [ 'restaurant_name', 'restaurant_id' ] )
                                                                                                                   .mean()
                                                                                                                   .sort_values( [ 'aggregate_rating', 'restaurant_id' ], ascending = [ False, True ] )
                                                                                                                   .reset_index()
                                                                                                                   .head(50) )
df_aux[ 'restaurant_name' ][0]

'Kanaat Lokantası'

##### 10. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a menor média de avaliação?


In [187]:
df_aux = ( df.loc[ ( df[ 'cuisines' ] == 'Home-made' ), [ 'aggregate_rating', 'restaurant_name', 'restaurant_id' ] ].groupby( [ 'restaurant_name', 'restaurant_id' ] )
                                                                                                                   .mean()
                                                                                                                   .sort_values( [ 'aggregate_rating', 'restaurant_id' ], ascending = True )
                                                                                                                   .reset_index()
                                                                                                                   .head(50) )
df_aux[ 'restaurant_name' ][0]

'GurMekan Restaurant'

##### 11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas pessoas?


In [198]:
df_aux = ( df.loc[ :, [ 'average_cost_for_two', 'cuisines' ] ].groupby( [ 'cuisines' ] )
                                                              .mean()
                                                              .sort_values( [ 'average_cost_for_two' ], ascending=False )
                                                              .reset_index() )

df_aux[ 'cuisines' ][0]

'Modern Australian'

##### 12. Qual o tipo de culinária que possui a maior nota média?


In [199]:
df_aux = ( df.loc[ :, [ 'aggregate_rating', 'cuisines' ] ].groupby( [ 'cuisines' ] )
                                                          .mean()
                                                          .sort_values( [ 'aggregate_rating' ], ascending=False )
                                                          .reset_index() )

df_aux[ 'cuisines' ][0]

'Others'

##### 13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas?

In [200]:
df_aux = ( df.loc[ ( df[ 'has_online_delivery' ] == 1 ) & ( df[ 'is_delivering_now' ] == 1 ), [ 'restaurant_id', 'cuisines' ] ].groupby( [ 'cuisines' ] )
                                                                                                                               .count()
                                                                                                                               .sort_values( [ 'restaurant_id' ], ascending=False )
                                                                                                                               .reset_index() )

df_aux[ 'cuisines' ][0]

'North Indian'